<a href="https://colab.research.google.com/github/hellokenta1024/notebooks/blob/master/Course_3_Week_2_Lesson_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# NOTE: PLEASE MAKE SURE YOU ARE RUNNING THIS IN A PYTHON3 ENVIRONMENT

import tensorflow as tf
print(tf.__version__)

# This is needed for the iterator over the data
# But not necessary if you have TF 2.0 installed
#!pip install tensorflow==2.0.0-beta0


#tf.enable_eager_execution()

#!pip install -q tensorflow-datasets

2.0.0-beta0


In [0]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


In [0]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [0]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)



In [19]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])
print(padded[1])
print(decode_review(padded[1]))
print(training_sentences[1])

[  59   15    4    1  341    5 5111   11   28    1   78  692   33 5155
    5   26 2897    8    8  268   26  504 1374   35  589 1884  807  990
    5  413  113   32  175  656    9 6414 3418  478   98   31   14    4
    1 5582    5 1302   13   99   29  197 8355   42 6666   15    2 1342
    1    9    4   96   27   14    4 5348    1    3    1   27   99   29
  631    3  649    9    2  170 4166   27    1 3150    1 7629 6589 3860
    3    1   17 5349    3 3488   10  200   29    4  749  259  889   19
   27   14    4 5863  559    8    8    3   10    7    2  209   13    7
   37  408 1026   38   26 7525   32    2]
b as a <OOV> fan of dickens i have <OOV> been disappointed by adaptations of his novels br br although his works presented an extremely accurate re telling of human life at every level in victorian britain throughout them all was a <OOV> thread of humour that could be both playful or sarcastic as the narrative <OOV> in a way he was a literary <OOV> and <OOV> he could be serious and hilar

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [14]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

W0702 13:04:45.658703 140392619165568 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 5s 197us/sample - loss: 0.5205 - accuracy: 0.7209 - val_loss: 0.3539 - val_accuracy: 0.8452
Epoch 2/10
25000/25000 [==============================] - 5s 186us/sample - loss: 0.2489 - accuracy: 0.9041 - val_loss: 0.3679 - val_accuracy: 0.8371
Epoch 3/10
25000/25000 [==============================] - 5s 185us/sample - loss: 0.0988 - accuracy: 0.9737 - val_loss: 0.4473 - val_accuracy: 0.8286
Epoch 4/10
25000/25000 [==============================] - 5s 185us/sample - loss: 0.0286 - accuracy: 0.9964 - val_loss: 0.5236 - val_accuracy: 0.8281
Epoch 5/10
25000/25000 [==============================] - 5s 185us/sample - loss: 0.0118 - accuracy: 0.9984 - val_loss: 0.5725 - val_accuracy: 0.8269
Epoch 6/10
25000/25000 [==============================] - 5s 185us/sample - loss: 0.0053 - accuracy: 0.9994 - val_loss: 0.6298 - val_accuracy: 0.8254
Epoch 7/10
25000/25000 [==========================

In [15]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [0]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [0]:


try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [18]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[11], [], [1431], [967], [4], [1537], [1537], [4740], [], [790], [2015], [11], [2922], [2191], [], [790], [2015], [11], [579], [], [11], [579], [], [4], [1783], [4], [4507], [11], [2922], [1277], [], [], [2015], [1005], [2922], [967], [579], [790], []]
